In [1]:
import numpy as np
import global_vars_go as gvg

# def switch_player_perspec(arry):
#     arry = arry.reshape(gvg.board_size, gvg.board_size, gvg.board_channels)
#     for i in range(len(arry)):
#         for j in range(len(arry[i])):
#             tmp = arry[i][j][gvg.player_channel]
#             arry[i][j][gvg.player_channel] = arry[i][j][gvg.bot_channel]
#             arry[i][j][gvg.bot_channel] = tmp
#     return arry

def setup_board(game):
    preboard, plays = sgf_moves.get_setup_and_moves(game)
    rpreboard = preboard.board
    board = np.zeros((gvg.board_size, gvg.board_size))
    if len(plays) < 1: # Return an empty board if the game has no moves
        return board
    if plays[0][0] == "b":
        color_stone = gvg.bot_in
    else:
        color_stone = gvg.player_in
    for i in range(len(rpreboard)):
        for j in range(len(rpreboard[i])):
            if rpreboard[i][j] == "b":
                board[i][j] = color_stone

    return board

In [2]:
import glob
import os
import sys
from sgfmill.sgfmill import sgf, sgf_moves
import numpy
import board as go_board

num_games = 3000
kifuPath = "../kifu/"

print("Loading game data...")

X = []
Y = []

i = 0
for filename in glob.glob(os.path.join(kifuPath, "*.sgf")):
    if i < num_games:
        with open(filename, "rb") as f:
            game = sgf.Sgf_game.from_bytes(f.read())
            board = setup_board(game)
            for node in game.get_main_sequence():
                board = -board
                node_move = node.get_move()[1]
                if node_move is not None:
                    X.append(np.copy(board))
                    move_board = np.zeros((19, 19))
                    move_board[node_move[0]][node_move[1]] = 1
                    Y.append(move_board)
                    board = go_board.make_move(board, node_move, gvg.bot_in, gvg.player_in, True)
    i += 1

X = np.array(X).reshape(-1, 19, 19, 1)
Y = np.array(Y).reshape(-1, 19 * 19)
testX = X[-4000:]
testY = Y[-4000:]
X = X[:-4000]
Y = Y[:-4000]

print("Done loading games")

Loading game data...
Done loading games


In [3]:
# Source: https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_mnist.py

# # -*- coding: utf-8 -*-

# """ Convolutional Neural Network for MNIST dataset classification task.

# References:
#     Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
#     learning applied to document recognition." Proceedings of the IEEE,
#     86(11):2278-2324, November 1998.

# Links:
#     [MNIST Dataset] http://yann.lecun.com/exdb/mnist/

# """

# from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# Building convolutional network
network = input_data(shape=[None, 19, 19, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
#network = max_pool_2d(network, 2)
#network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
#network = max_pool_2d(network, 2)
#network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 19*19, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')

In [4]:
# Training1
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path="checkpoints/convnet_snap.ckpt")
model.fit({'input': X}, {'target': Y}, n_epoch=3,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=3000, show_metric=True, run_id='convnet_go')

Training Step: 27851  | total loss: 4.22074 | time: 163.748s
| Adam | epoch: 003 | loss: 4.22074 - acc: 0.1791 -- iter: 594112/594141
Training Step: 27852  | total loss: 4.25405 | time: 164.782s
| Adam | epoch: 003 | loss: 4.25405 - acc: 0.1706 | val_loss: 4.45226 - val_acc: 0.1583 -- iter: 594141/594141
--
INFO:tensorflow:C:\Users\Edan\Desktop\GoHeuristics\tflearnImplementation\checkpoints\convnet_snap.ckpt-27852 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\Edan\Desktop\GoHeuristics\tflearnImplementation\checkpoints\convnet_snap.ckpt-27852 is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
import glob
import os
import sys
from sgfmill.sgfmill import sgf, sgf_moves
import numpy
import board as go_board

num_games_test = 500
kifuPath = "../kifu/"

print("Loading game data...")

X2 = []
Y2 = []

i = 0
for filename in glob.glob(os.path.join(kifuPath, "*.sgf")):
    if num_games < i < num_games + num_games_test:
        with open(filename, "rb") as f:
            game = sgf.Sgf_game.from_bytes(f.read())
            board = setup_board(game)
            for node in game.get_main_sequence():
                board = -board
                node_move = node.get_move()[1]
                if node_move is not None:
                    X2.append(np.copy(board))
                    move_board = np.zeros((gvg.board_size, gvg.board_size))
                    move_board[node_move[0]][node_move[1]] = gvg.filled
                    Y2.append(move_board)
                    board = go_board.make_move(board, node_move, gvg.bot_in, gvg.player_in, True)
    i += 1

X2 = np.array(X2).reshape(-1, 19, 19, 1)
Y2 = np.array(Y2).reshape(-1, 19 * 19)

print("Done loading games")

Loading game data...


In [ ]:
model.evaluate(X2, Y2)